# NYC Covid-19 Analysis

Thanks to the NYC Department of Health and Mental Hygine, daily COVID-19 data on testings, areas being tested, age, sex, etc. have been made available for people to see. Toady, I will just be trying to look at some initial analysis on whats been happening throughout the New York Area. Show any trends throughout daily information as well as any hotspots in the area. 

## Data Sets Provided

Daily updates are provided for the data at https://github.com/nychealth/coronavirus-data

# Import 

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Cleaning and Merging 

We first scrapped data on the NYC boroughs and all zipcodes within them from:
https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm 

Mannual cleaning in excel was peformed to list each zipcode with its borough.

In [48]:
bz = pd.read_csv('Bourgh_zipcodes.csv')

In [49]:
bz.head()

,Borough,ZIP Codes
0,Bronx,"10453, 10457, 10460, 10458, 10467, 10468, 1045..."
1,Brooklyn,"11212, 11213, 11216, 11233, 11238, 11209, 1121..."
2,Manhattan,"10026, 10027, 10030, 10037, 10039, 10001, 1001..."
3,Queens,"11361, 11362, 11363, 11364, 11354, 11355, 1135..."
4,Staten Island,"10302, 10303, 10310, 10306, 10307, 10308, 1030..."


In [50]:
zips = [bz['ZIP Codes'][i].strip().split(',') for i in range(0,5)]

In [51]:
bronx = pd.DataFrame({'BOROUGH': 'Bronx', 'zip_codes':zips[0]})

In [52]:
brook = pd.DataFrame({'BOROUGH': 'Brooklyn', 'zip_codes':zips[1]})

In [53]:
manhattan = pd.DataFrame({'BOROUGH': 'Manhattan', 'zip_codes':zips[2]})

In [54]:
queens = pd.DataFrame({'BOROUGH': 'Queens', 'zip_codes':zips[3]})

In [55]:
si = pd.DataFrame({'BOROUGH': 'Staten Island', 'zip_codes':zips[4]})

In [56]:
borough_zips = pd.concat([bronx, brook, manhattan, queens, si])

In [57]:
borough_zips['zip_codes'] = borough_zips['zip_codes'].astype('int64')

Since information is made daily for us, these functions created below will update our csv's so that we will have the latest data made availble for our visualizations. 

In [58]:
old_date = '4:11'
new_date = '4:12'

In [59]:
#Function for updating positive test by Borough and Zipcode
def update_testing (new_date):
    test_zip = pd.read_csv(f'coronavirus-data {new_date}/tests-by-zcta.csv')
    test_zip.drop(0, inplace = True)
    test_zip['MODZCTA'] = test_zip['MODZCTA'].astype('int64')
    test_zip.rename(columns = {'MODZCTA': 'zip_codes'}, inplace = True)
    testing = pd.merge(left = borough_zips,
        right = test_zip,
        how = 'inner',
        on = 'zip_codes')
    boro = pd.read_csv(f'coronavirus-data {new_date}/boro.csv')
    boro.drop(5,inplace = True)
    boro.loc[:, 'BOROUGH_GROUP'] = testing['BOROUGH'].unique()
    boro.rename(columns = {'BOROUGH_GROUP': 'BOROUGH'}, inplace = True)
    testing_borough = pd.merge(left = testing,
        right = boro,
        how = 'inner',
        on = 'BOROUGH')
    return testing_borough.to_csv('master_tests.csv',index = False)

In [60]:
update_testing(new_date)

In [61]:
#Function for updating Summary
def update_summary (old_date,new_date): #remember in 4:2 format for april 2nd()
    old_sum = pd.read_csv(f'master_summary_{old_date}.csv')#.T #- remove # before t if needed to update
    #incase file is overwritten, rewrite using the following code 
    #old_sum.columns = old_sum.iloc[0]
    #old_sum.drop('Cases:', inplace = True)
    #old_sum['Cases'] = old_sum.index
    #old_sum['As of:'] = pd.to_datetime(old_sum['As of:'])

    new_sum = pd.read_csv(f'coronavirus-data {new_date}/summary.csv').T
    new_sum.columns = new_sum.iloc[0]
    new_sum.drop('Cases:', inplace = True)
    new_sum['Cases'] = new_sum.index
    new_sum['As of:'] = pd.to_datetime(new_sum['As of:'])
    
    merged = pd.concat([old_sum,new_sum])
    merged.reset_index(inplace = True)
    merged.drop(columns = 'index', inplace = True)
    date = [i.split(' ') for i in summary['As of:']]
    merged['dates'] = [date[i][0] for i in range(len(date))]
    del merged['As of:']
    merged.set_index(['dates'], inplace = True)
    merged = merged.sort_values(by = 'dates')
    merged.to_csv(f'master_summary_{new_date}.csv', index = False)
     
    return merged.head()


In [63]:
update_summary(old_date,new_date)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


,As of:,Cases,Deaths:,NYC confirmed deaths:,Total hospitalized*:
0,2020-04-02 17:00:00,49707,1562.0,NaN,10590
1,2020-04-05 17:00:00,64955,2472.0,NaN,14205
2,2020-04-06 17:00:00,68776,2738.0,NaN,15333
3,2020-03-26 17:00:00,23112,364.0,NaN,4712
4,2020-03-27 17:00:00,26697,450.0,NaN,5039


In [65]:
#formatting casses by day file
case = pd.read_csv(f'coronavirus-data {new_date}/case-hosp-death.csv')
case['DATE_OF_INTEREST'] = pd.to_datetime(case['DATE_OF_INTEREST'])
case.set_index(['DATE_OF_INTEREST'], inplace = True)

In [66]:
case.head()

,NEW_COVID_CASE_COUNT,HOSPITALIZED_CASE_COUNT,DEATH_COUNT
DATE_OF_INTEREST,,,
2020-03-02,1,12.0,NaN
2020-03-03,3,10.0,NaN
2020-03-04,10,13.0,NaN
2020-03-05,2,18.0,NaN
2020-03-06,8,17.0,NaN


In [67]:
case['NCCC_diff1'] = case['NEW_COVID_CASE_COUNT'].diff(1)

Created new column to show the difference in day to day NEW_COVID_CASE_COUNT

In [68]:
case['DC_diff1'] = case['DEATH_COUNT'].diff(1)

Created new column to show the difference in day to day DEATH_COUNT

In [84]:
case.head()

,NEW_COVID_CASE_COUNT,HOSPITALIZED_CASE_COUNT,DEATH_COUNT,NCCC_diff1,DC_diff1,NCCC_diff7
DATE_OF_INTEREST,,,,,,
2020-03-02,1,12.0,NaN,NaN,NaN,NaN
2020-03-03,3,10.0,NaN,2.0,NaN,NaN
2020-03-04,10,13.0,NaN,7.0,NaN,NaN
2020-03-05,2,18.0,NaN,-8.0,NaN,NaN
2020-03-06,8,17.0,NaN,6.0,NaN,NaN


In [70]:
case.to_csv(f'coronavirus-data {new_date}/case-hosp-death.csv')

Fucntion created to regularly update age with new data daily provided

In [71]:
# Fucntion created to regularly update age with new data daily provided
def update_age (old_date,new_date):
    old_age = pd.read_csv(f'master_age_{old_date}.csv')
    #old_age['date'] = f'{old_date[0]}/{old_date[2]}/2020'
    #old_age['date'] = pd.to_datetime(old_age['date'])
    
    new_age = pd.read_csv(f'coronavirus-data {new_date}/by-age.csv')
    new_age['date'] = f'{new_date[0]}/{new_date[2] + new_date[3]}/2020'
    new_age['date'] = pd.to_datetime(new_age['date'])
    
    merged = pd.concat([old_age,new_age])
    merged.reset_index(inplace = True)
    merged.drop(columns = 'index', inplace = True)
    merged['date'] = pd.to_datetime(merged['date'])
    merged.to_csv(f'master_age_{new_date}.csv', index = False)
    
    return merged.head()

In [72]:
update_age(old_date, new_date)

,AGE_GROUP,COVID_CASE_RATE,HOSPITALIZED_CASE_RATE,DEATH_RATE,date
0,0-17 years,51.02,5.10,0.06,2020-04-02
1,18-44 years,602.82,53.92,2.67,2020-04-02
2,45-64 years,851.00,192.70,18.58,2020-04-02
3,65-74 years,871.81,328.99,53.78,2020-04-02
4,75 and older years,879.56,442.98,130.35,2020-04-02


Fucntion created to regularly update Gender with new data daily provided

In [74]:
def update_gender (old_date,new_date):
    old_sex = pd.read_csv(f'master_sex_{old_date}.csv')
    #old_sex['date'] = f'{old_date[0]}/{old_date[2]}/2020'
    #old_sex['date'] = pd.to_datetime(old_sex['date'])
    
    new_sex = pd.read_csv(f'coronavirus-data {new_date}/by-sex.csv')
    new_sex['date'] = f'{new_date[0]}/{new_date[2] + new_date[3]}/2020'
    new_sex['date'] = pd.to_datetime(new_sex['date'])
    
    merged = pd.concat([old_sex, new_sex])
    merged.reset_index(inplace = True)
    merged.drop(columns = 'index', inplace = True)
    merged['date'] = pd.to_datetime(merged['date'])
    merged.to_csv(f'master_sex_{new_date}.csv', index = False)
    
    return merged.head()

In [75]:
update_gender(old_date, new_date)

,SEX_GROUP,COVID_CASE_RATE,HOSPITALIZED_CASE_RATE,DEATH_RATE,date
0,Female,511.86,97.71,13.41,2020-04-02
1,Male,677.86,157.14,24.24,2020-04-02
2,Citywide total,591.84,126.09,18.60,2020-04-02
3,Female,673.99,130.08,20.94,2020-04-05
4,Male,880.09,211.86,38.69,2020-04-05


All Files are now updated and ready for upload in Tableau